# Install all packages

In [1]:
!pip install -q pandas great-expectations openai streamlit plotly pyyaml kaggle
print("✅ Main packages installed.")


✅ Main packages installed.


# Configue API Kaggle

In [2]:
#Set up Kaggle API using Colab Secrets
import os
from google.colab import userdata

print("🔐 Setting up Kaggle API from Colab Secrets...")

# Create the .kaggle directory
os.makedirs('/root/.kaggle', exist_ok=True)

# Read credentials from Secrets
kaggle_username = userdata.get('KAGGLE_USERNAME')
kaggle_key = userdata.get('KAGGLE_KEY')

# Create the kaggle.json content
kaggle_config = f'{{"username":"{kaggle_username}","key":"{kaggle_key}"}}'

# Write the config file
with open('/root/.kaggle/kaggle.json', 'w') as f:
    f.write(kaggle_config)

# Set secure permissions
os.chmod('/root/.kaggle/kaggle.json', 600)

print("✅ Kaggle API securely configured from Secrets.")

🔐 Setting up Kaggle API from Colab Secrets...
✅ Kaggle API securely configured from Secrets.


# Data Ingestion & Exploration

In [3]:
# Download and extract the dataset
!kaggle datasets download -d olistbr/brazilian-ecommerce
!unzip -o brazilian-ecommerce.zip -d ./olist_data
print("📥 Dataset downloaded and extracted.")

Dataset URL: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce
License(s): CC-BY-NC-SA-4.0
brazilian-ecommerce.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  brazilian-ecommerce.zip
  inflating: ./olist_data/olist_customers_dataset.csv  
  inflating: ./olist_data/olist_geolocation_dataset.csv  
  inflating: ./olist_data/olist_order_items_dataset.csv  
  inflating: ./olist_data/olist_order_payments_dataset.csv  
  inflating: ./olist_data/olist_order_reviews_dataset.csv  
  inflating: ./olist_data/olist_orders_dataset.csv  
  inflating: ./olist_data/olist_products_dataset.csv  
  inflating: ./olist_data/olist_sellers_dataset.csv  
  inflating: ./olist_data/product_category_name_translation.csv  
📥 Dataset downloaded and extracted.


In [4]:
# Load and explore the main tables
import pandas as pd
import numpy as np

# Load key tables
customers = pd.read_csv('./olist_data/olist_customers_dataset.csv')
orders = pd.read_csv('./olist_data/olist_orders_dataset.csv')
order_items = pd.read_csv('./olist_data/olist_order_items_dataset.csv')
products = pd.read_csv('./olist_data/olist_products_dataset.csv')
order_reviews = pd.read_csv('./olist_data/olist_order_reviews_dataset.csv')

print("📊 Dataset Overview:")
print(f"Customers: {customers.shape}")
print(f"Orders: {orders.shape}")
print(f"Order Items: {order_items.shape}")
print(f"Products: {products.shape}")
print(f"Order Reviews: {order_reviews.shape}")
print("\n🔍 Sample from Orders table:")
print(orders.head())

📊 Dataset Overview:
Customers: (99441, 5)
Orders: (99441, 8)
Order Items: (112650, 7)
Products: (32951, 9)
Order Reviews: (99224, 7)

🔍 Sample from Orders table:
                           order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
2  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
3  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
4  ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   

  order_status order_purchase_timestamp    order_approved_at  \
0    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
1    delivered      2018-07-24 20:41:37  2018-07-26 03:24:27   
2    delivered      2018-08-08 08:38:49  2018-08-08 08:55:23   
3    delivered      2017-11-18 19:28:06  2017-11-18 19:45:59   
4    delivered      2018-02-13 21:18:39  2018-02-13 22:20:29   

  order_delive

In [5]:
# Create a unified dataset for analysis
# Merge key tables
df_merged = orders.merge(order_items, on='order_id') \
                  .merge(products, on='product_id') \
                  .merge(customers, on='customer_id') \
                  .merge(order_reviews[['order_id', 'review_score']],
                         on='order_id',
                         how='left')

print("🔄 Merged dataset shape:", df_merged.shape)
print("📋 Merged columns:", df_merged.columns.tolist())

🔄 Merged dataset shape: (113314, 27)
📋 Merged columns: ['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value', 'product_category_name', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state', 'review_score']


# Great Expectations for data validation

In [6]:
!pip uninstall -y great_expectations
!pip install great_expectations==0.18.12

Found existing installation: great-expectations 0.18.12
Uninstalling great-expectations-0.18.12:
  Successfully uninstalled great-expectations-0.18.12
  Using cached great_expectations-0.18.12-py3-none-any.whl.metadata (8.9 kB)
Using cached great_expectations-0.18.12-py3-none-any.whl (5.4 MB)


In [ ]:
import json
import yaml
import great_expectations as gx


print("🧪 Starting Great Expectations Data Quality Check (v0.18.12)...")

# 1. CREATE CONTEXT AND CHECK FOR EXISTING SUITE
context = gx.get_context()
expectation_suite_name = "olist_quality_suite"

# Remove the suite if it exists from a previous run (to avoid conflicts)
if expectation_suite_name in context.list_expectation_suite_names():
    context.delete_expectation_suite(expectation_suite_name)
    print(f"⚠️  Deleted existing suite: '{expectation_suite_name}'")

# Create a fresh expectation suite
suite = context.add_expectation_suite(expectation_suite_name)
print(f"✅ Created expectation suite: '{expectation_suite_name}'")

# 2. CREATE VALIDATOR USING RuntimeBatchRequest (Correct pattern for v0.18)
from great_expectations.core.batch import RuntimeBatchRequest

batch_request = RuntimeBatchRequest(
    datasource_name="my_temp_datasource",
    data_connector_name="default_runtime_data_connector",
    data_asset_name="merged_orders",
    runtime_parameters={"batch_data": df_merged},
    batch_identifiers={"run_id": "colab_run_1"},
)

# You must add a simple datasource configuration for the runtime batch request to work.
# This uses a minimal in-memory Pandas datasource.
datasource_config = {
    "name": "my_temp_datasource",
    "class_name": "Datasource",
    "execution_engine": {"class_name": "PandasExecutionEngine"},
    "data_connectors": {
        "default_runtime_data_connector": {
            "class_name": "RuntimeDataConnector",
            "batch_identifiers": ["run_id"],
        }
    },
}
context.add_datasource(**datasource_config)
print("✅ Added temporary in-memory datasource.")

# Get the validator
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name=expectation_suite_name,
)

# 3. ADD YOUR EXPECTATIONS (Data Quality Rules)
print("📝 Adding data quality expectations...")
validator.expect_column_values_to_not_be_null(column="order_id")
validator.expect_column_values_to_be_unique(column="order_id")
validator.expect_column_values_to_be_between(
    column="price",
    min_value=0.01,
    max_value=10000
)
validator.expect_column_values_to_be_in_set(
    column="order_status",
    value_set=["delivered", "shipped", "canceled", "unavailable", "processing", "invoiced"]
)

# Save expectations
validator.save_expectation_suite(discard_failed_expectations=False)
print("✅ Expectations saved.")

# 4. CREATE AND RUN A SIMPLE CHECKPOINT
checkpoint_config = {
    "name": "olist_colab_checkpoint",
    "config_version": 1.0,
    "class_name": "SimpleCheckpoint",
    "validations": [
        {
            "batch_request": batch_request,
            "expectation_suite_name": expectation_suite_name
        }
    ]
}

checkpoint = context.test_yaml_config(yaml.dump(checkpoint_config))
results = checkpoint.run()

# 5. PROCESS AND DISPLAY RESULTS
# Extract results from the slightly different checkpoint result structure
run_results = list(results.run_results.values())[0]
validation_result = run_results["validation_result"]

print("\n" + "="*60)
print("DATA QUALITY REPORT (v0.18.12)")
print("="*60)

if validation_result["success"]:
    print("✅ All critical data quality checks passed!")
else:
    print("⚠️  Some checks failed:")
    for result in validation_result["results"]:
        if not result["success"]:
            col = result["expectation_config"]["kwargs"].get("column", "N/A")
            exp_type = result["expectation_config"]["expectation_type"]
            # Show a sample of unexpected values if available
            unexpected_list = result.get("result", {}).get("partial_unexpected_list", [])
            if unexpected_list:
                # Format list for display, show max 3 items
                display_list = str(unexpected_list[:3])
                if len(unexpected_list) > 3:
                    display_list = display_list[:-1] + ", ...]"
                print(f"   • Column '{col}' ({exp_type}): e.g., {display_list}")
            else:
                print(f"   • Column '{col}': {exp_type}")

# 6. SAVE DETAILED REPORT
report_filename = "data_quality_report_018.json"
with open(report_filename, "w") as f:
    json.dump(validation_result, f, indent=2)

print(f"\n📄 Detailed report saved to: {report_filename}")
print(f"✅ Data Quality Check Complete for GX v0.18.12!")

  return datetime.utcnow().replace(tzinfo=utc)

INFO:great_expectations.data_context.types.base:Created temporary directory '/tmp/tmps1eiv_c1' for ephemeral docs site


🧪 Starting Great Expectations Data Quality Check (v0.18.12)...


  return datetime.utcnow().replace(tzinfo=utc)



✅ Created expectation suite: 'olist_quality_suite'
✅ Added temporary in-memory datasource.


  return datetime.utcnow().replace(tzinfo=utc)

  warnings.warn(



...

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

  return datetime.utcnow().replace(tzinfo=utc)

  warnings.warn(



Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

  return datetime.utcnow().replace(tzinfo=utc)



✅ Expectations saved.
Attempting to instantiate class from config...
	Instantiating as a SimpleCheckpoint, since class_name is SimpleCheckpoint


# OpenAI to analyze product reviews

In [ ]:
import openai
import time
from google.colab import userdata

print("🤖 Starting LLM-based Sentiment Analysis...")

# Securely load OpenAI API key
openai.api_key = userdata.get('OPENAI_API_KEY')

def analyze_review_sentiment(review_text):
    """Use GPT to analyze sentiment and extract keywords"""
    if not review_text or pd.isna(review_text):
        return None

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful data analyst. Always respond with valid JSON."},
                {"role": "user", "content": f"""Analyze this product review and return JSON with:
                1. sentiment (positive/negative/neutral)
                2. primary_emotion (one word)
                3. keywords (array of 3-5 keywords)
                Review: "{review_text[:500]}" """}
            ],
            temperature=0.1,
            max_tokens=150
        )

        result = response.choices[0].message.content
        # Clean JSON response
        if result.startswith("```json"):
            result = result[7:-3]
        elif result.startswith("```"):
            result = result[3:-3]

        return eval(result)  # Convert string to dict

    except Exception as e:
        print(f"Error analyzing review: {e}")
        return None

# Sample analysis on a subset
print("Analyzing sample reviews (first 5)...")
sample_reviews = order_reviews['review_comment_message'].dropna().head(5).tolist()

for i, review in enumerate(sample_reviews):
    analysis = analyze_review_sentiment(review)
    print(f"\nReview {i+1}:")
    print(f"Text: {review[:100]}...")
    print(f"Analysis: {analysis}")
    time.sleep(0.5)  # Avoid rate limits

print("\n✅ Sentiment analysis complete!")

# Create a comprehensive Streamlit dashboard

In [ ]:
%%writefile /content/ecommerce_dashboard.py
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

# Page config
st.set_page_config(
    page_title="Brazilian E-commerce Dashboard",
    page_icon="🇧🇷",
    layout="wide"
)

# Title
st.title("🇧🇷 Brazilian E-commerce Analytics Dashboard")
st.markdown("Interactive analysis of Olist's e-commerce data")

# Load data (cached for performance)
@st.cache_data
def load_data():
    customers = pd.read_csv('./olist_data/olist_customers_dataset.csv')
    orders = pd.read_csv('./olist_data/olist_orders_dataset.csv')
    order_items = pd.read_csv('./olist_data/olist_order_items_dataset.csv')
    products = pd.read_csv('./olist_data/olist_products_dataset.csv')

    # Merge data
    df = orders.merge(order_items, on='order_id') \
               .merge(products, on='product_id') \
               .merge(customers, on='customer_id')

    # Convert dates
    df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
    df['order_month'] = df['order_purchase_timestamp'].dt.to_period('M').astype(str)

    return df

df = load_data()

# Sidebar filters
st.sidebar.header("📊 Filters")
date_range = st.sidebar.date_input(
    "Date Range",
    value=[df['order_purchase_timestamp'].min().date(),
           df['order_purchase_timestamp'].max().date()]
)

selected_states = st.sidebar.multiselect(
    "Customer States",
    options=df['customer_state'].unique(),
    default=df['customer_state'].unique()[:3]
)

price_range = st.sidebar.slider(
    "Price Range (R$)",
    float(df['price'].min()),
    float(df['price'].max()),
    (0.0, 500.0)
)

# Apply filters
filtered_df = df[
    (df['order_purchase_timestamp'].dt.date >= date_range[0]) &
    (df['order_purchase_timestamp'].dt.date <= date_range[1]) &
    (df['customer_state'].isin(selected_states)) &
    (df['price'] >= price_range[0]) &
    (df['price'] <= price_range[1])
]

# KPI Metrics
st.header("📈 Key Performance Indicators")
col1, col2, col3, col4 = st.columns(4)

with col1:
    total_orders = filtered_df['order_id'].nunique()
    st.metric("Total Orders", f"{total_orders:,}")

with col2:
    total_revenue = filtered_df['price'].sum()
    st.metric("Total Revenue", f"R$ {total_revenue:,.2f}")

with col3:
    avg_order_value = filtered_df.groupby('order_id')['price'].sum().mean()
    st.metric("Avg Order Value", f"R$ {avg_order_value:,.2f}")

with col4:
    unique_customers = filtered_df['customer_id'].nunique()
    st.metric("Unique Customers", f"{unique_customers:,}")

# Visualizations
st.header("📊 Sales Analysis")

tab1, tab2, tab3 = st.tabs(["Trends", "Geography", "Products"])

with tab1:
    # Monthly sales trend
    monthly_sales = filtered_df.groupby('order_month').agg({
        'order_id': 'nunique',
        'price': 'sum'
    }).reset_index()

    fig_trend = go.Figure()
    fig_trend.add_trace(go.Scatter(
        x=monthly_sales['order_month'],
        y=monthly_sales['price'],
        mode='lines+markers',
        name='Revenue',
        line=dict(color='#FF4B4B')
    ))

    fig_trend.update_layout(
        title="Monthly Revenue Trend",
        xaxis_title="Month",
        yaxis_title="Revenue (R$)",
        hovermode='x unified'
    )

    st.plotly_chart(fig_trend, use_container_width=True)

with tab2:
    # Sales by state
    state_sales = filtered_df.groupby('customer_state').agg({
        'order_id': 'nunique',
        'price': 'sum'
    }).reset_index()

    fig_map = px.bar(
        state_sales,
        x='customer_state',
        y='price',
        color='order_id',
        title="Revenue by Customer State",
        labels={'price': 'Revenue (R$)', 'order_id': 'Number of Orders'},
        color_continuous_scale='Viridis'
    )

    st.plotly_chart(fig_map, use_container_width=True)

with tab3:
    # Top products
    top_products = filtered_df.groupby('product_category_name').agg({
        'order_id': 'nunique',
        'price': 'sum'
    }).nlargest(10, 'price').reset_index()

    fig_products = px.treemap(
        top_products,
        path=['product_category_name'],
        values='price',
        color='order_id',
        title="Top 10 Product Categories by Revenue",
        color_continuous_scale='RdBu'
    )

    st.plotly_chart(fig_products, use_container_width=True)

# Data preview
st.header("🔍 Filtered Data Preview")
st.dataframe(
    filtered_df[[
        'order_id', 'order_purchase_timestamp',
        'customer_state', 'product_category_name',
        'price', 'freight_value'
    ]].head(20),
    use_container_width=True
)

# Download button
st.sidebar.header("📥 Export")
csv = filtered_df.to_csv(index=False)
st.sidebar.download_button(
    label="Download Filtered Data as CSV",
    data=csv,
    file_name="filtered_ecommerce_data.csv",
    mime="text/csv"
)

st.sidebar.markdown("---")
st.sidebar.info("Dashboard created for Dadosfera Technical Test")

# Launch the Streamlit app from Colab

In [ ]:
print("🚀 Launching Streamlit Dashboard...")
print("This will generate a public URL. Copy it to access your dashboard.")
print("-" * 50)

# Run Streamlit in the background
get_ipython().system_raw('streamlit run /content/ecommerce_dashboard.py --server.port 8501 --server.enableCORS false &')

# Use localtunnel to expose the app
!npm install -q localtunnel
import time
time.sleep(5)  # Wait for Streamlit to start

print("\nYour public dashboard URL:")
!npx localtunnel --port 8501 2>&1 | grep -o "https://[^ ]*" | head -1